In [2]:
import memory_profiler as mem_prof
from time import time
from functools import wraps
import numbers
import os

## Closures

### Global accessible by the function

In [50]:
msg = 'global'
del msg
def outer():
    try:
        print(msg)
    except Exception as e:
        print(f'{e.__class__.__name__}, {e}')
    
outer()

NameError, name 'msg' is not defined


In [51]:
msg = 'global'
del msg
def outer():
    try:
        print(msg)
    except NameError:
        print(f'No such variable msg')
        
outer()

No such variable msg


In [56]:
msg = 'global'
del msg
def outer():
    if 'msg' in locals():
        print(msg)
    else:
        raise NameError('No such variable msg')
outer()

NameError: No such variable msg

In [59]:
msg = 'global'
def outer():
    try:
        print(f'variable msg in outer function is {msg}')
    except NameError:
        print(f'No such variable msg')
        
outer()

variable msg in outer function is global


### The UnboundLocalError

In [105]:
msg = 'global'

def outer():
    msg = msg + ' appended in outer' ## same result with msg += ' appended in outer'
    
outer()

UnboundLocalError: local variable 'msg' referenced before assignment

### Problem not the same with printing the global msg

In [106]:
msg = 'global'

def outer():
    print(msg)
    
outer()

global


### Just add global to make any appending changes

In [109]:
msg = 'global'

def outer():
    global msg
    msg = msg + ' appended in outer' ## same result with msg += ' appended in outer'
    print(msg)
    
outer()
print(msg)

global appended in outer
global appended in outer


### Global accessible by both inner and outer


In [69]:
msg = 'global'

def outer():
    print(f'outer msg is {msg}')
        
    def inner():
        print(f'inner msg is {msg}')
            
    inner()
    
outer()

outer msg is global
inner msg is global


### Global won't be overrided, rather added to locals when you assign it inside the outer function

In [78]:
msg = 'global'

def outer():
    msg = 'outer'
    print(f'outer msg is {msg}')
        
    def inner():
        print(f'inner msg is {msg}')
            
    inner()
    
outer()
print(msg)

outer msg is outer
inner msg is outer
global


### But can be overrided by explicitly specifying global

In [79]:
msg = 'global'

def outer():
    global msg
    msg = 'outer'
    print(f'outer msg is {msg}')
        
    def inner():
        print(f'inner msg is {msg}')
            
    inner()
    
outer()
print(msg)

outer msg is outer
inner msg is outer
outer


### Preference taken first for local

In [71]:
msg = 'global'

def outer():
    msg = 'outer'
    print(f'outer msg is {msg}')
        
    def inner():
        print(f'inner msg is {msg}')
            
    inner()

outer()

outer msg is outer
inner msg is outer


### Although it allows overriding of the msg defined in outer (not the global)

In [89]:
msg = 'global'

def outer():
    msg = 'outer'
    print(f'outer msg is {msg}')
        
    def inner():
        print(f'inner msg is {msg}')
            
    inner()

outer()
print(msg)

outer msg is outer
inner msg is outer
global


### Need to use the global msg in inner function but not the outer? ==> add global

In [90]:
msg = 'global'

def outer():
    msg = 'outer'
    print(f'outer msg is {msg}')
        
    def inner():
        global msg
        print(f'inner msg is {msg}')
            
    inner()

outer()
print(msg)

outer msg is outer
inner msg is global
global


### Same UnboundLocalError when appending to outer msg


In [114]:
msg = 'global'

def outer():
    msg = 'outer'
    print(f'outer msg is {msg}')
        
    def inner():
        msg += ' inner'
        print(f'inner msg is {msg}')
            
    inner()
    print(f'outer msg after redefining in inner is {msg}')

outer()
print(msg)

outer msg is outer


UnboundLocalError: local variable 'msg' referenced before assignment

### Want to access and override the outer msg ==> add nonlocal 

In [113]:
msg = 'global'

def outer():
    msg = 'outer'
    print(f'outer msg is {msg}')
        
    def inner():
        nonlocal msg
        msg = 'inner'
        print(f'inner msg is {msg}')
            
    inner()
    print(f'outer msg after redefining in inner is {msg}')

outer()
print(msg)

outer msg is outer
inner msg is inner
outer msg after redefining in inner is inner
global


### But surprisingly this works with assigning keys to a dictionary or lists defined in outer without specifying nonlocal? => Woah!

In [123]:
def outer():
    msg = {'outer': 1}
    print(f'outer function msg is {msg}')
    
    def inner():
        msg['outer'] = 2
        print(f'inner function msg is {msg}')
    
    inner()
    print(f'msg in outer after assigning a key in inner is {msg}')
    
outer()

outer function msg is {'outer': 1}
inner function msg is {'outer': 2}
msg in outer after assigning a key in inner is {'outer': 2}


### The reason behind this behaviour
refer [here](https://stackoverflow.com/questions/14323817/global-dictionaries-dont-need-keyword-global-to-modify-them)

Assigning a string or an integer is ambigous i.e. it could either be referring to a global or a local variable.
So python resorts to defining a new local variable rather than overriding it.
But would raise an error if it is appended to

In [125]:
msg = 'global'

def outer():
    msg = 'outer'
    print(msg)
    
outer()
print(f'msg after running outer is {msg}')

outer
msg after running outer is global


In [127]:
msg = 'global'

def outer():
    msg += ' appending in outer'
    print(msg)

try:
    outer()
except UnboundLocalError as e:
    print(e)

print(f'msg after running outer is {msg}')

local variable 'msg' referenced before assignment
msg after running outer is global


But assigning a key to a dictionary or appending to a list is unambigous as the line dictvar['key1'] += 1 or listvar.append(1) can only be referring to global (or nonlocal) for it to not throw an error.

In [132]:
msg = dict()
msg['global'] = 'global'
print(f'msg in global before calling outer() is {msg}')

def outer():
    msg['global'] = f"{msg['global']} appended in outer"
    print(msg)

outer()
print(f'msg in global after calling outer() is {msg}')

msg in global before calling outer() is {'global': 'global'}
{'global': 'global appended in outer'}
msg in global after calling outer() is {'global': 'global appended in outer'}


The dictionary insertion is not an assignment rather an insertion which calls \_\_setitem_\_ method the dict

In [135]:
dictvar = dict()
dictvar.__setitem__('key', 'val')
dictvar

{'key': 'val'}

### Listvars can be referenced and assigned via the index inside the outer

In [146]:
glob_lst = [1, 2 , 3]

def outer():
    glob_lst[0] = 0
    
outer()
print(glob_lst)

[0, 2, 3]


### Generators and functions

In [183]:
def squared_lst(lst):
    sq_lst = list()
    for num in lst:
        sq_lst.append(num**2)
        
    return sq_lst
        
lst = [1, 2, 3 ,4]
print(lst)
sq_list = squared_lst(lst)
print(sq_list)

[1, 2, 3, 4]
[1, 4, 9, 16]


In [184]:
def squared_gen(lst):
    for num in lst:
        yield num**2

In [185]:
sq_gen = squared_gen(lst)
sq_gen

<generator object squared_gen at 0x7f52f5cb7938>

In [186]:
for num in sq_gen:
    print(num)

1
4
9
16


In [166]:
sq_gen = squared_lst_gen(lst)
print(next(sq_gen))
print(next(sq_gen))
print(next(sq_gen))
print(next(sq_gen))
print(next(sq_gen))

1
4
9
16


StopIteration: 

### Approximately how under the hood looping works

In [169]:
lst = [1, 2, 3 ,4]
iter_lst = iter(lst)

while True:
    try:
        print(next(iter_lst))
    except StopIteration:
        break

1
2
3
4


### Infact range() is a generator

### Let's time it for a large list of num and see the memory usage

In [194]:
def timer(func):
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        t1 = time()
        rv = func(*args, **kwargs)
        t2 = time()
        print(f'Elapsed: {t2-t1}s')
        return rv
    
    return wrapper

In [210]:
def mem_use(func):
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        m1 = mem_prof.memory_usage()[0]
        rv = func(*args, **kwargs)
        m2 = mem_prof.memory_usage()[0]
        print(f'Memory occupied: {m2-m1}MB')
        
        return rv
    
    return wrapper

In [212]:
@timer
@mem_use
def squared_lst(lst):
    sq_lst = list()
    for num in lst:
        sq_lst.append(num**2)
        
    return sq_lst

In [225]:
@timer
@mem_use
def squared_gen(lst):
    for num in lst:
        yield num**2

In [227]:
lst = list(range(10**7))

In [228]:
print('For the function returning the list','\n')
result_lst = squared_lst(lst)

For the function returning the list 

Memory occupied: 386.07421875MB
Elapsed: 3.149179220199585s


In [229]:
print('For the function returning the generator','\n')
result_gen = squared_gen(lst)

For the function returning the generator 

Memory occupied: 0.0MB
Elapsed: 0.20267915725708008s


### But this does not account for looping through times

In [231]:
%%time

for num in result_lst:
    pass

CPU times: user 184 ms, sys: 0 ns, total: 184 ms
Wall time: 183 ms


In [232]:
%%time 

for num in result_gen:
    pass

CPU times: user 2.75 s, sys: 0 ns, total: 2.75 s
Wall time: 2.75 s


### Let's delete the large lists that we created

In [234]:
try:
    del lst, result_lst, result_gen
except NameError:
    pass

### Some operations with classes

### Implement the __copy__ method for a class

#### A. Make a Point Class with the following rules
        1. Has x, y, z, origin
        2. Can be translated along the three axes
        3. Re-setting the origin translates the point's coords
        4. Can be multiplied, matrix-multiplied, added, and subtracted
        5. Comparative operators between instances
        6. Can remember states of translation. (Extra step: output json)
        

In [181]:
class Point():
    
    def __init__(self, x=0 , y=0, z=0, o=0):
        self.o = o       
        self.x = x 
        self.y = y
        self.z = z
        
    def getReferencePoint(self, other=None):
        new_point = Point(**self.__dict__)
        if other is None:
            new_point.o = 0
        elif type(other) is numbers.Number:
            new_point.o = other
        elif isinstance(other, self.__class__):
            new_point.o = other.o
            
        return new_point
            
    
    def __setattr__(self, attr, val):
        if attr is 'o':
            if (hasattr(self, 'x') and 
                hasattr(self, 'y') and 
                hasattr(self, 'z')):
                
                diff = val - self.o
                self.translate(x=diff, y=diff, z=diff, method='by')
                
        object.__setattr__(self, attr, val)  ## Avoid RecursionError
            
    @property
    def coords(self):
        return [self.x, self.y, self.z, self.o]
        
        
    def translate(self, x=0, y=0, z=0, method='by'):
        for attr in ['x', 'y', 'z']:
            if method is 'by':
                self.__setattr__(attr, self.__getattribute__(attr) + eval(attr))
            else:
                self.__setattr__(attr, self.__getattribute__(attr))
                
    
    def __len__(self):
        return sum([(self.__getattribute__(attr) - self.o)**2 for attr in ['x', 'y', 'z']])
        
        
    def __repr__(self):
        return f'Point(x={self.x}, y={self.y}, z={self.z}, o={self.o})'
        
    def __add__(self, other):
        new_point = Point(**other.__dict__)
        new_point.o = self.o
        for attr in ['x', 'y', 'z']:
            new_point.__setattr__(attr, self.__getattribute__(attr) + new_point.__getattribute__(attr))
        
        return new_point
    
    def __sub__(self, other):
        new_point = Point(o=self.o)
        for attr in ['x', 'y', 'z']:
            new_point.__setattr__(attr, self.__getattribute__(attr) - other.__getattribute__(attr))
        
        return new_point 
    
    def __mul__(self, other):
        
        if isinstance(other, numbers.Number):
            new_point = Point(o=self.o)
            for attr in ['x', 'y', 'z']:
                new_point.__setattr__(attr, self.__getattribute__(attr)*other)
                
        elif type(other) is list:
            new_point = Point(o=self.o)
            for i, attr in enumerate(['x', 'y', 'z']):
                new_point.__setattr__(attr, self.__getattribute__(attr)*other[i])
                
        elif isinstance(other, Point):
            new_point = Point(**other.__dict__)
            new_point.o = self.o
            for attr in ['x', 'y', 'z']:
                new_point.__setattr__(attr, self.__getattribute__(attr)*new_point.__getattribute__(attr))
                                
        return new_point

#### 1. Has x, y, z, origin

In [182]:
p1 = Point(1, 2, 3)
allattr = zip(('x', 'y', 'z', 'o'), (p1.x, p1.y, p1.z, p1.o))

for attr, val in allattr:
    print(attr, val)

x 1
y 2
z 3
o 0


In [183]:
p1 = Point(1, 2, 3, 1)
print(p1)

Point(x=1, y=2, z=3, o=1)


#### 2. Can be translated along the three axes

In [184]:
p1 = Point(1, 2, 3)
print('Before ==>', p1)

trans_dict = {'x': 1, 'y': 1, 'z': 1}
print()
for trans in trans_dict.items():
    print('Translating along', trans[0], 'by', trans[1])
    t2d = {trans[0]: trans[1]}
    p1.translate(**t2d, method='by')
    print('\t', p1, '\n')
    p1 = Point(1, 2, 3)

Before ==> Point(x=1, y=2, z=3, o=0)

Translating along x by 1
	 Point(x=2, y=2, z=3, o=0) 

Translating along y by 1
	 Point(x=1, y=3, z=3, o=0) 

Translating along z by 1
	 Point(x=1, y=2, z=4, o=0) 



#### 3. Resetting the origin translates the point's coords

In [185]:
p1 = Point(1, 2, 3, 1)
print('Before ==>', p1, '\n')
old_o = p1.o

p1.o = 5
print(f'Set origin from {old_o} to {p1.o}')
print('\t', p1)

Before ==> Point(x=1, y=2, z=3, o=1) 

Set origin from 1 to 5
	 Point(x=5, y=6, z=7, o=5)


####  4. Can be multiplied, matrix-multiplied, added, and subtracted

4.1 Multiply

In [186]:
p1 = Point(1, 2, 3, 1)
p2 = Point(4, 5, 6, -2)
print('p1 ==>', p1)
print('p2 ==>', p2)
print()
print('p1*p2 ==>', p1*p2)
print('p2*p1 ==>', p2*p1)

p1 ==> Point(x=1, y=2, z=3, o=1)
p2 ==> Point(x=4, y=5, z=6, o=-2)

p1*p2 ==> Point(x=7, y=16, z=27, o=1)
p2*p1 ==> Point(x=-8, y=-5, z=0, o=-2)


4.2 Matrix Multiply

4.3 Add

In [187]:
p1 = Point(1, 2, 3, 1)
p2 = Point(4, 5, 6, -2)
print('p1 ==>', p1)
print('p2 ==>', p2)
print()
print('p1+p2 ==>', p1+p2)
print('p2+p1 ==>', p2+p1)

p1 ==> Point(x=1, y=2, z=3, o=1)
p2 ==> Point(x=4, y=5, z=6, o=-2)

p1+p2 ==> Point(x=8, y=10, z=12, o=1)
p2+p1 ==> Point(x=2, y=4, z=6, o=-2)


4.4 Subtract